# Titre 
explain

Nous avons fait le choix de nous concentrer sur l'action de NVIDIA car ... 

# A. Récupération des données 

Nous utilisons Yahoo Finance (via yfinance) pour récupérer les séries boursières car c’est une source simple d’accès, largement utilisée pour des données daily OHLCV (Open, High, Low, Close, Volume), et crédible. 

L’objectif est de construire une base fiable et reproductible en combinant trois sources :
- NVDA (Yahoo Finance) : données quotidiennes sur NVIDIA (OHLCV, Adj Close, dividendes et splits). C’est la base principale du projet.
- ETFs (Yahoo Finance) : séries quotidiennes SPY / QQQ / SOXX pour capturer le contexte marché / tech / semi-conducteurs et distinguer les chocs “systématiques” des effets spécifiques à NVDA.
- FRED (pandas-datareader) : indicateurs macro-financiers VIX (régime de volatilité) et taux US 10 ans (conditions financières), utilisés comme variables de régime.

Enfin, toutes les séries sont alignées sur un calendrier unique (les dates de cotation de NVDA) afin de garantir la cohérence temporelle. Nous sauvegardons systématiquement les données en deux étapes (raw puis processed) pour assurer la reproductibilité : le projet peut être rejoué à l’identique sans dépendre d’un nouvel appel aux sources externes, ce qui fournit un backup si la disponibilité ou la structure des données venait à évoluer.

In [30]:
#importation de modules 
import pandas as pd
from pandas_datareader import data as web
from pathlib import Path
import yfinance as yf 

In [31]:
### import de data et set up 
#période de 10 ans 
start_date = "2015-01-01"
end_date   = "2024-12-31"

#Sélection des .. d'intérêt  
etf_tickers  = ["SPY", "QQQ", "SOXX"]
fred_series  = ["VIXCLS", "DGS10"]
ticker = "NVDA"

#Gestion des paths ...
RAW_DIR = Path("data/raw")
RAW_DIR.mkdir(parents=True, exist_ok=True)
nvda_raw_path = RAW_DIR / "nvda_stock_data_raw.csv"
etf_raw_path  = RAW_DIR / "etf_stock_data_raw.csv"
fred_raw_path = RAW_DIR / "fred_macro_raw.csv"

In [32]:
## Téléchargement des données de NVIDIA

#Si le fichier raw existe déjà, on le recharge (évite de retélécharger à chaque exécution)
if nvda_raw_path.exists():
    nvda_data_2 = pd.read_csv(nvda_raw_path, parse_dates=["Date"])

#Sinon, on télécharge NVDA via Yahoo
else:
    nvda_data_2 = yf.download(
        ticker,
        start=start_date,
        end=end_date,
        interval="1d",
        auto_adjust=False,
        actions=True,
        progress=False
    ).reset_index()

    nvda_data_2 = nvda_data_2.rename(columns={c: f"{c}_{ticker}" for c in nvda_data_2.columns if c != "Date"})
    
    #Save en cache pour la reproductibilité 
    nvda_data_2.to_csv(nvda_raw_path, index=False)

#Affihcage de contrôle de la cellue 
nvda_data_2.head()

,Date,Adj Close,Close,Dividends,High,Low,Open,Stock Splits,Volume
0,NaT,NVDA,NVDA,NVDA,NVDA,NVDA,NVDA,NVDA,NVDA
1,2015-01-02,0.483011394739151,0.503250002861023,0.0,0.5070000290870667,0.49524998664855957,0.503250002861023,0.0,113680000
2,2015-01-05,0.4748532474040985,0.4947499930858612,0.0,0.5047500133514404,0.4925000071525574,0.503250002861023,0.0,197952000
3,2015-01-06,0.4604565501213074,0.47975000739097595,0.0,0.4959999918937683,0.4792500138282776,0.49549999833106995,0.0,197764000
4,2015-01-07,0.45925670862197876,0.47850000858306885,0.0,0.48750001192092896,0.47699999809265137,0.4832499921321869,0.0,321808000


In [33]:
## Téléchargment de ETFs + FRED

#ETFs : (pareil que pour NVDA)
if etf_raw_path.exists():
    etf_data = pd.read_csv(etf_raw_path, parse_dates=["Date"])
else:
    etf = yf.download(
        etf_tickers,
        start=start_date,
        end=end_date,
        interval="1d",
        auto_adjust=False,
        actions=True,
        progress=False,
        group_by="column"
    )
    etf.columns = [f"{field}_{tic}" for field, tic in etf.columns]
    etf_data = etf.reset_index()
    etf_data.to_csv(etf_raw_path, index=False)

#FRED(VIXCLS, DGS10) : (pareil que pour NVDA)
if fred_raw_path.exists():
    fred_data = pd.read_csv(fred_raw_path, parse_dates=["DATE"])
else:
    fred_data = web.DataReader(fred_series, "fred", start_date, end_date).reset_index()
    fred_data.to_csv(fred_raw_path, index=False)

#Affihcages de contrôle de la cellue 
fred_data.head()
etf_data.head()

,Date,Adj Close_QQQ,Adj Close_SOXX,Adj Close_SPY,Capital Gains_QQQ,Capital Gains_SOXX,Capital Gains_SPY,Close_QQQ,Close_SOXX,Close_SPY,...,Low_SPY,Open_QQQ,Open_SOXX,Open_SPY,Stock Splits_QQQ,Stock Splits_SOXX,Stock Splits_SPY,Volume_QQQ,Volume_SOXX,Volume_SPY
0,2015-01-02,94.784454,27.559072,170.589645,0.0,0.0,0.0,102.940002,30.936666,205.429993,...,204.179993,103.760002,31.100000,206.380005,0.0,0.0,0.0,31314600,663000,121465900
1,2015-01-05,93.394089,27.039421,167.508850,0.0,0.0,0.0,101.430000,30.353333,201.720001,...,201.350006,102.489998,30.806667,204.169998,0.0,0.0,0.0,36521300,619500,169632600
2,2015-01-06,92.141785,26.436640,165.931046,0.0,0.0,0.0,100.070000,29.676666,199.820007,...,198.860001,101.580002,30.373333,202.089996,0.0,0.0,0.0,66205500,1123800,209151400
3,2015-01-07,93.329597,26.697945,167.998764,0.0,0.0,0.0,101.360001,29.969999,202.309998,...,200.880005,100.730003,29.846666,201.419998,0.0,0.0,0.0,37577400,721200,125346700
4,2015-01-08,95.115906,27.490774,170.979965,0.0,0.0,0.0,103.300003,30.860001,205.899994,...,203.990005,102.220001,30.273333,204.009995,0.0,0.0,0.0,40212600,633000,147217800


In [34]:
## Création de la base finale  
#TODO : annoter

nvda_cols = ["Date"] + [c for c in nvda_data_2.columns if c.endswith("_NVDA")]
nvda_base = nvda_data_2[nvda_cols].copy()


#On a remarqué que le nom de variable de la date est différent entre FRED et NVDA
#On renomme simplement cette variable pour merge : "DATE" -> "Date"
if "DATE" in fred_data.columns and "Date" not in fred_data.columns:
    fred_data = fred_data.rename(columns={"DATE": "Date"})

#?
for df in [nvda_base, etf_data, fred_data]:
    df["Date"] = pd.to_datetime(df["Date"])
    df.sort_values("Date", inplace=True)

#Indexation selon les dates de cotations de NVDA
nvda_idx = nvda_base.set_index("Date")
etf_idx  = etf_data.set_index("Date")
fred_idx = fred_data.set_index("Date")


#?
etf_aligned  = etf_idx.reindex(nvda_idx.index)
fred_aligned = fred_idx.reindex(nvda_idx.index).ffill()

#Marge par left join 
nvda_data_2 = (
    nvda_idx
    .join(etf_aligned, how="left")
    .join(fred_aligned, how="left")
    .reset_index()
)


#Sauvegarde 

PROCESSED_DIR = Path("data/processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

processed_parquet = PROCESSED_DIR / "dataset_merged_nvda_etf_fred.parquet"
processed_csv     = PROCESSED_DIR / "dataset_merged_nvda_etf_fred.csv"

try:
    nvda_data_2.to_parquet(processed_parquet, index=False)
    print("Saved parquet:", processed_parquet)
except ImportError:
    nvda_data_2.to_csv(processed_csv, index=False)
    print("Parquet engine missing -> saved CSV instead:", processed_csv)

#Affichage de contrôle 

print("Merged nvda_data shape:", nvda_data_2.shape)
print(nvda_data_2.isna().sum().sort_values(ascending=False).head(10))
nvda_data_2.head()

Parquet engine missing -> saved CSV instead: data/processed/dataset_merged_nvda_etf_fred.csv
Merged nvda_data shape: (2516, 30)
Date                 1
Adj Close_QQQ        1
Volume_SPY           1
Volume_SOXX          1
Volume_QQQ           1
Stock Splits_SPY     1
Stock Splits_SOXX    1
Stock Splits_QQQ     1
Open_SPY             1
Open_SOXX            1
dtype: int64


,Date,Adj Close_QQQ,Adj Close_SOXX,Adj Close_SPY,Capital Gains_QQQ,Capital Gains_SOXX,Capital Gains_SPY,Close_QQQ,Close_SOXX,Close_SPY,...,Open_SOXX,Open_SPY,Stock Splits_QQQ,Stock Splits_SOXX,Stock Splits_SPY,Volume_QQQ,Volume_SOXX,Volume_SPY,VIXCLS,DGS10
0,2015-01-02,94.784454,27.559072,170.589645,0.0,0.0,0.0,102.940002,30.936666,205.429993,...,31.100000,206.380005,0.0,0.0,0.0,31314600.0,663000.0,121465900.0,17.79,2.12
1,2015-01-05,93.394089,27.039421,167.508850,0.0,0.0,0.0,101.430000,30.353333,201.720001,...,30.806667,204.169998,0.0,0.0,0.0,36521300.0,619500.0,169632600.0,19.92,2.04
2,2015-01-06,92.141785,26.436640,165.931046,0.0,0.0,0.0,100.070000,29.676666,199.820007,...,30.373333,202.089996,0.0,0.0,0.0,66205500.0,1123800.0,209151400.0,21.12,1.97
3,2015-01-07,93.329597,26.697945,167.998764,0.0,0.0,0.0,101.360001,29.969999,202.309998,...,29.846666,201.419998,0.0,0.0,0.0,37577400.0,721200.0,125346700.0,19.31,1.96
4,2015-01-08,95.115906,27.490774,170.979965,0.0,0.0,0.0,103.300003,30.860001,205.899994,...,30.273333,204.009995,0.0,0.0,0.0,40212600.0,633000.0,147217800.0,17.01,2.03


Final checks and DL les bases cleans 

- données manquantes ?
- recherche de doublons de journée ?
- colonnes utilisées à garder 

Checks 
- describe 
- corr
- bonus : sweekness

Fin de la récupération et du traitement des données. 